## 1. Import Libraries
Importing all libraries in the cell below:

In [11]:
#Import numpy
import numpy as np

#Import tensorflow
import tensorflow as tf

#Import API to load MNIST dataset 
from tensorflow.examples.tutorials.mnist import input_data

#Import tflearn for Experiment
import tensorflow.contrib.learn as tflearn

#Import learn runner to run the experiment
from tensorflow.contrib.learn.python.learn import learn_runner

#Import metrics for evaluation with experiment
import tensorflow.contrib.metrics as metrics


## 2. Setting up the Dataset
In this step we will import the MNIST dataset via Tensorflow API itself.

In [12]:
#Load the dataset
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 3. Defining the Input Functions for Training and Evaluation
Here we define the training and evaluation input functions that requires the data (x) and their respective labels (y), number of epochs and the shuffle argument. Rest arguments are set to default.

In [13]:
# Define training input func.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x": np.array(mnist.train.images)},
                    y=np.array(mnist.train.labels,dtype=np.int32),
                    num_epochs=100,
                    shuffle=True)

In [14]:
# Define validation input func.
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x":np.array(mnist.validation.images)},
                    y=np.array(mnist.validation.labels,dtype=np.int32),
                    num_epochs=1,
                    shuffle=False)

## 4. Setting up the Classifier
Now we define what our feature columns are and initialize our Deep Neural Network Classifier. 3 hidden layers are specified and their respective neurons as well. The number of classes and the output directory is also specified in the end.

In [15]:
# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("x",dimension=784)]

In [16]:
#Specify Model Directory
output_dir = 'mnist_exp_model'
#Init the classifier
classifier = tflearn.DNNClassifier(
                        feature_columns=feature_columns,
                        hidden_units=[10, 32, 10],
                        n_classes=10,
                        model_dir=output_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_master': '', '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000187B9518710>, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_model_dir': 'mnist_exp_model', '_task_type': None, '_evaluation_master': '', '_num_worker_replicas': 0, '_tf_random_seed': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_id': 0, '_save_checkpoints_secs': 600}


## 5. Setting up and Running the Experiment
Now we will specifiy the requirments of an Experiment that takes input func, eval func and the classifier. Lastly, we run the experiment we outlined via learn runner. 

In [18]:
def exp_fn(output_dir):
    return tflearn.Experiment(
        
        #Provide the classifier we just made
        estimator = classifier,
        
        #Provide the training input func. we made
        train_input_fn = train_input_fn,
        
        #Provide the eval input func. we made
        eval_input_fn = eval_input_fn,
        
        #Specify the eval metrics to learn on
        eval_metrics={
            'accuracy': tflearn.MetricSpec(
                metric_fn      = metrics.streaming_accuracy,
                prediction_key = tflearn.PredictionKey.CLASSES
            ),
            'precision': tflearn.MetricSpec(
                metric_fn      = metrics.streaming_precision,
                prediction_key = tflearn.PredictionKey.CLASSES
            ),
            'recall': tflearn.MetricSpec(
                metric_fn      = metrics.streaming_recall,
                prediction_key = tflearn.PredictionKey.CLASSES
            )
        }
    )
#Run the Experiment!
model = learn_runner.run(exp_fn, output_dir)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into mnist_exp_model\model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-12-24-12:40:59
INFO:tensorflow:Restoring parameters from mnist_exp_model\model.ckpt-1
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:

## 6. Evaluating our Model with Test Inputs
In this section we will define our test inputs so as to evaluate our experimented model via accuracy score.


In [19]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
                  x={"x": np.array(mnist.test.images)},
                  y=np.array(mnist.test.labels,dtype=np.int32),
                  num_epochs=1,
                  shuffle=False,
                  )

In [20]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

INFO:tensorflow:Starting evaluation at 2017-12-24-12:47:23
INFO:tensorflow:Restoring parameters from mnist_exp_model\model.ckpt-42969
INFO:tensorflow:Finished evaluation at 2017-12-24-12:47:24
INFO:tensorflow:Saving dict for global step 42969: accuracy = 0.9436, global_step = 42969, loss = 0.19414
